In [1]:
import numpy as np
import pandas as pd
import pdb
import os
import math

In [2]:
#requirements:
metadata = pd.read_csv('/csc/epitkane/projects/muat/extfile/metadata_icgc_pcawg.tsv',sep='\t',index_col=0) 
dictMutation = pd.read_csv('/csc/epitkane/projects/muat/extfile//dictMutation.csv',index_col=0)
dictChpos = pd.read_csv('/csc/epitkane/projects/muat/extfile/dictChpos.csv',index_col=0)
dictGES = pd.read_csv('/csc/epitkane/projects/muat/extfile/dictGES.csv',index_col=0)

In [3]:
data_dir = '/csc/epitkane/projects/EpicSV/data/master_bedgraphs/'

In [4]:
filename = 'master_ChromHMM_145_epigenomes_hg19.bedg.gz'

In [5]:
epigene = pd.read_csv(data_dir + filename, sep='\t')

In [6]:
epigen_token = pd.read_csv('epigen_token.csv',index_col=0)

In [14]:
epigene

,chrom,start,end,liver_ENCDO776UMN,liver_ENCDO832GKP,liver_ENCDO232KQV,right_lobe_of_liver_ENCDO793LXB,hepatic_stellate_cell_ENCDO282AAA,hepatocyte_ENCDO102AAA,Hep-G2_ENCDO000AAC,...,T-cell_CD-8+_ENCDO273TST,T-cell_CD-8+_ENCDO579CCQ,WERI-Rb-1_ENCDO000ADT,K562_ENCDO000AAD,KBM-7_ENCDO763VKT,NCI-H929_ENCDO220OYR,RPMI-8226_ENCDO624OGB,KMS-11_ENCDO133EXU,testis_ENCDO845WKR,testis_ENCDO451RUA
0,chr1,0,9800,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
1,chr1,9800,10000,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Het,Quies,Quies,Quies,Quies,Quies,Quies
2,chr1,10000,10400,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Het,Quies,Quies,Quies,Quies,Quies,Quies
3,chr1,10400,13000,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Het,Quies,Quies,Quies,Quies,Quies,Quies
4,chr1,13000,13200,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,ZNF/Rpts,Quies,Quies,Quies,Quies,Quies,Quies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687936,chrX,154929200,154929400,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
8687937,chrX,154929400,154930000,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
8687938,chrX,154930000,154930400,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
8687939,chrX,154930400,155270400,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies


In [25]:
dictEpigen = epigen_token['epigen_state'].to_dict()

In [26]:
dictEpigen = {v: k for k, v in dictEpigen.items()}

In [27]:
dictEpigen

{'Missing': 0,
 'EnhA1': 1,
 'EnhA2': 2,
 'EnhBiv': 3,
 'EnhG1': 4,
 'EnhG2': 5,
 'EnhWk': 6,
 'Het': 7,
 'Quies': 8,
 'ReprPC': 9,
 'ReprPCWk': 10,
 'TssA': 11,
 'TssBiv': 12,
 'TssFlnk': 13,
 'TssFlnkD': 14,
 'TssFlnkU': 15,
 'Tx': 16,
 'TxWk': 17,
 'ZNF/Rpts': 18}

In [9]:
list_col = epigene.iloc[:,3:].columns

In [10]:
epigene.iloc[:,3:]

,liver_ENCDO776UMN,liver_ENCDO832GKP,liver_ENCDO232KQV,right_lobe_of_liver_ENCDO793LXB,hepatic_stellate_cell_ENCDO282AAA,hepatocyte_ENCDO102AAA,Hep-G2_ENCDO000AAC,HuH-7_ENCDO000ACU,prostate_ENCDO845WKR,prostate_ENCDO451RUA,...,T-cell_CD-8+_ENCDO273TST,T-cell_CD-8+_ENCDO579CCQ,WERI-Rb-1_ENCDO000ADT,K562_ENCDO000AAD,KBM-7_ENCDO763VKT,NCI-H929_ENCDO220OYR,RPMI-8226_ENCDO624OGB,KMS-11_ENCDO133EXU,testis_ENCDO845WKR,testis_ENCDO451RUA
0,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
1,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Het,Quies,Quies,Quies,Quies,Quies,Quies
2,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Het,Quies,Quies,Quies,Quies,Quies,Quies
3,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Het,Quies,Quies,Quies,Quies,Quies,Quies
4,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,ZNF/Rpts,Quies,Quies,Quies,Quies,Quies,Quies
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687936,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
8687937,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
8687938,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies
8687939,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,...,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies,Quies


In [33]:
colnm = list_col[0]

In [34]:
colnm

'liver_ENCDO776UMN'

In [37]:
for i in list_col:
    epigene = epigene.replace({i: dictEpigen})

In [28]:
sourcedir = '/csc/epitkane/projects/PCAWG20191001/data/modified_data/raw/indelmeisubsvperclass/'

In [29]:
all_class = os.listdir(sourcedir)

In [30]:
pcawg_dir = '/csc/epitkane/projects/PCAWG20191001/data/modified_data/train/allclasses/simplified_data/'

In [31]:
tokenized_data = '/csc/epitkane/projects/PCAWG20191001/data/modified_data/train/allclasses/tokenized_data/'

In [39]:
epigene.to_csv('/csc/epitkane/projects/PCAWG20191001/data/modified_data/train/allclasses/epigenetic_token.tsv',sep='\t')

In [20]:
epigenetic_token = pd.read_csv('/csc/epitkane/projects/PCAWG20191001/data/modified_data/train/allclasses/epigenetic_token.tsv',sep='\t')

In [22]:
epigenetic_token.columns

Index(['Unnamed: 0', 'chrom', 'start', 'end', 'liver_ENCDO776UMN',
       'liver_ENCDO832GKP', 'liver_ENCDO232KQV',
       'right_lobe_of_liver_ENCDO793LXB', 'hepatic_stellate_cell_ENCDO282AAA',
       'hepatocyte_ENCDO102AAA',
       ...
       'T-cell_CD-8+_ENCDO273TST', 'T-cell_CD-8+_ENCDO579CCQ',
       'WERI-Rb-1_ENCDO000ADT', 'K562_ENCDO000AAD', 'KBM-7_ENCDO763VKT',
       'NCI-H929_ENCDO220OYR', 'RPMI-8226_ENCDO624OGB', 'KMS-11_ENCDO133EXU',
       'testis_ENCDO845WKR', 'testis_ENCDO451RUA'],
      dtype='object', length=149)

In [23]:
epigenetic_token.drop(columns=['Unnamed: 0'])

,chrom,start,end,liver_ENCDO776UMN,liver_ENCDO832GKP,liver_ENCDO232KQV,right_lobe_of_liver_ENCDO793LXB,hepatic_stellate_cell_ENCDO282AAA,hepatocyte_ENCDO102AAA,Hep-G2_ENCDO000AAC,...,T-cell_CD-8+_ENCDO273TST,T-cell_CD-8+_ENCDO579CCQ,WERI-Rb-1_ENCDO000ADT,K562_ENCDO000AAD,KBM-7_ENCDO763VKT,NCI-H929_ENCDO220OYR,RPMI-8226_ENCDO624OGB,KMS-11_ENCDO133EXU,testis_ENCDO845WKR,testis_ENCDO451RUA
0,chr1,0,9800,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
1,chr1,9800,10000,8,8,8,8,8,8,8,...,8,8,8,7,8,8,8,8,8,8
2,chr1,10000,10400,8,8,8,8,8,8,8,...,8,8,8,7,8,8,8,8,8,8
3,chr1,10400,13000,8,8,8,8,8,8,8,...,8,8,8,7,8,8,8,8,8,8
4,chr1,13000,13200,8,8,8,8,8,8,8,...,8,8,8,18,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687936,chrX,154929200,154929400,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8687937,chrX,154929400,154930000,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8687938,chrX,154930000,154930400,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8687939,chrX,154930400,155270400,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [24]:
epigenetic_token

,Unnamed: 0,chrom,start,end,liver_ENCDO776UMN,liver_ENCDO832GKP,liver_ENCDO232KQV,right_lobe_of_liver_ENCDO793LXB,hepatic_stellate_cell_ENCDO282AAA,hepatocyte_ENCDO102AAA,...,T-cell_CD-8+_ENCDO273TST,T-cell_CD-8+_ENCDO579CCQ,WERI-Rb-1_ENCDO000ADT,K562_ENCDO000AAD,KBM-7_ENCDO763VKT,NCI-H929_ENCDO220OYR,RPMI-8226_ENCDO624OGB,KMS-11_ENCDO133EXU,testis_ENCDO845WKR,testis_ENCDO451RUA
0,0,chr1,0,9800,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
1,1,chr1,9800,10000,8,8,8,8,8,8,...,8,8,8,7,8,8,8,8,8,8
2,2,chr1,10000,10400,8,8,8,8,8,8,...,8,8,8,7,8,8,8,8,8,8
3,3,chr1,10400,13000,8,8,8,8,8,8,...,8,8,8,7,8,8,8,8,8,8
4,4,chr1,13000,13200,8,8,8,8,8,8,...,8,8,8,18,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8687936,8687936,chrX,154929200,154929400,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8687937,8687937,chrX,154929400,154930000,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8687938,8687938,chrX,154930000,154930400,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
8687939,8687939,chrX,154930400,155270400,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [ ]:
for i in range(37,len(all_class)):
    all_samples = os.listdir(sourcedir + all_class[i])
    
    pcawg_histology = all_class[i]
    os.makedirs(pcawg_dir + pcawg_histology, exist_ok=True)
    for j in range(0,len(all_samples)):
        read_sample = pd.read_csv(sourcedir + all_class[i] + '/' + all_samples[j])
        
        somatic_files_persamples = read_sample
        
        pcawg_sample = all_samples[j][:-4]
        
        #1) simplifing mutation information
        simplified_mutation = []
        for k in range(0,len(somatic_files_persamples)):
            onemut = somatic_files_persamples.iloc[k]
            trp = onemut['seq'][int(len(onemut['seq'])/2)-1:int(len(onemut['seq'])/2)+2]
            ps = math.floor(onemut['pos'] / 1000000)
            chompos = str(onemut['chrom']) + '_' + str(ps)
            rt = onemut['rt']
            ges = str(onemut['genic']) + '_' + str(onemut['exonic']) + '_' + str(onemut['strand'])
            try:
                typ = dictMutation.loc[dictMutation['triplet']==trp]['mut_type'].values[0]
            except:
                typ = 'Neg'

            tuple_onerow = (trp,chompos,rt,ges,typ)
            simplified_mutation.append(tuple_onerow)
        pd_new = pd.DataFrame(simplified_mutation)
        pd_new.columns = ['seq','chrompos','rt','ges','mut_type'] 
        #save simplified mutation
        #new_files = 'new_' + pcawg_sample + '.csv'
        
        #2) tokenization from simplified mutation files
        mergetriplet = pd_new.merge(dictMutation, left_on='seq', right_on='triplet', how='left',suffixes=('', '_y'))
        mergetriplet.drop(mergetriplet.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergeges = mergetriplet.merge(dictGES, left_on='ges', right_on='ges', how='left',suffixes=('', '_y'))
        mergeges.drop(mergeges.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergeges = mergeges.rename(columns={"token": "gestoken"})
        mergechrompos = mergeges.merge(dictChpos, left_on='chrompos', right_on='chrompos', how='left',suffixes=('', '_y'))
        mergechrompos.drop(mergechrompos.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergechrompos = mergechrompos.rename(columns={"token": "postoken"})
        token_data = mergechrompos[['triplettoken','postoken','gestoken','rt','mut_type']]
        
        SNVonly = token_data.loc[token_data['mut_type']=='SNV']
        SNVonly = SNVonly.drop(columns=['mut_type'])
        MNVonly = token_data.loc[token_data['mut_type']=='MNV']
        MNVonly = MNVonly.drop(columns=['mut_type'])
        indelonly = token_data.loc[token_data['mut_type']=='indel']
        indelonly = indelonly.drop(columns=['mut_type'])
        MEISVonly = token_data.loc[token_data['mut_type'].isin(['MEI','SV'])]
        Negonly = token_data.loc[token_data['mut_type']=='Normal']
        Negonly = Negonly.drop(columns=['mut_type'])
        
        SNVonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'SNV_' + pcawg_sample + '.csv')
        MNVonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'MNV_' + pcawg_sample + '.csv')
        indelonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'indel_' + pcawg_sample + '.csv')
        MEISVonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'MEISV_' + pcawg_sample + '.csv')
        Negonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'Normal_' + pcawg_sample + '.csv')
        pd_count = pd.DataFrame([len(SNVonly),len(MNVonly),len(indelonly),len(MEISVonly),len(Negonly)])
        pd_count.to_csv(pcawg_dir + pcawg_histology + '/' + 'count_' + pcawg_sample + '.csv')
        
        print(pcawg_histology + '/' + pcawg_sample)  

In [13]:
for i in range(0,len(all_class)):
    all_samples = os.listdir(sourcedir + all_class[i])
    print(all_class[i])

    pcawg_histology = all_class[i]
    os.makedirs(pcawg_dir + pcawg_histology, exist_ok=True)
    for j in range(0,len(all_samples)):
        read_sample = pd.read_csv(sourcedir + all_class[i] + '/' + all_samples[j])

        somatic_files_persamples = read_sample

        pcawg_sample = all_samples[j][:-4]

        #1) simplifing mutation information
        simplified_mutation = []
        for k in range(0,len(somatic_files_persamples)):
            onemut = somatic_files_persamples.iloc[k]
            trp = onemut['seq'][int(len(onemut['seq'])/2)-1:int(len(onemut['seq'])/2)+2]
            ps = math.floor(onemut['pos'] / 1000000)
            chompos = str(onemut['chrom']) + '_' + str(ps)
            rt = onemut['rt']
            ges = str(onemut['genic']) + '_' + str(onemut['exonic']) + '_' + str(onemut['strand'])
            try:
                typ = dictMutation.loc[dictMutation['triplet']==trp]['mut_type'].values[0]
            except:
                typ = 'Neg'
            tuple_onerow = (trp,chompos,rt,ges,typ,onemut['chrom'],onemut['pos'])
            simplified_mutation.append(tuple_onerow)
        pd_new = pd.DataFrame(simplified_mutation)
        pd_new.columns = ['seq','chrompos','rt','ges','mut_type','chrom','pos'] 
        #save simplified mutation
        new_files = pcawg_sample + '.csv'
        
        pd_new.to_csv(pcawg_dir + pcawg_histology + '/' + new_files)

Bladder-TCC
Bone-Osteosarc
Bone-Cart
Bone-Epith
Breast-AdenoCA
Breast-LobularCA
Breast-DCIS
Biliary-AdenoCA
Cervix-SCC
Cervix-AdenoCA
Lymph-CLL
Myeloid-MPN
Myeloid-MDS
Myeloid-AML
ColoRect-AdenoCA
Lymph-BNHL
Prost-AdenoCA
Eso-AdenoCA
Stomach-AdenoCA
CNS-GBM
Head-SCC
Kidney-ChRCC
Kidney-RCC
CNS-Oligo
Liver-HCC
Lung-AdenoCA
Lung-SCC
Lymph-NOS
Skin-Melanoma


/csc/epitkane/software/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Ovary-AdenoCA
Panc-AdenoCA
Panc-Endocrine
CNS-Medullo
CNS-PiloAstro
Bone-Leiomyo
Thy-AdenoCA
Uterus-AdenoCA


In [ ]:
for i in range(0,len(all_class)):
    all_samples = os.listdir(sourcedir + all_class[i])
    print(all_class[i])

    pcawg_histology = all_class[i]
    os.makedirs(tokenized_data + pcawg_histology, exist_ok=True)
    for j in range(0,len(all_samples)):
        pd_new = pd.read_csv(pcawg_dir + all_class[i] + '/' + all_samples[j],index_col=0)

        mergetriplet = pd_new.merge(dictMutation, left_on='seq', right_on='triplet', how='left',suffixes=('', '_y'))
        mergetriplet.drop(mergetriplet.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergeges = mergetriplet.merge(dictGES, left_on='ges', right_on='ges', how='left',suffixes=('', '_y'))
        mergeges.drop(mergeges.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergeges = mergeges.rename(columns={"token": "gestoken"})
        mergechrompos = mergeges.merge(dictChpos, left_on='chrompos', right_on='chrompos', how='left',suffixes=('', '_y'))
        mergechrompos.drop(mergechrompos.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
        mergechrompos = mergechrompos.rename(columns={"token": "postoken"})

        pcawg_sample = all_samples[j][:-4]

        #1) simplifing mutation information
        epigenetics_list = []
        pd_epiall = pd.DataFrame()
        for k in range(0,len(mergechrompos)):
            onemut = mergechrompos.iloc[k]
            chrom = onemut['chrom']
            pos = onemut['pos']
            all_chrom = epigene.loc[epigene['chrom']==str('chr'+chrom)]
            all_pos = all_chrom.loc[all_chrom['start']<=pos]
            get_epi = all_pos.loc[all_chrom['end']>=pos]

            if len(get_epi) == 0:
                pd_epi = pd.DataFrame(np.zeros(145),dtype=int).T
                pd_epi.columns = pd_epiall.columns
            else:
                pd_epi = pd.DataFrame(get_epi.iloc[0,3:]).T

            pd_epiall = pd_epiall.append(pd_epi)

        pd_epiall = pd_epiall.reset_index(drop=True)
        mergechrompos = mergechrompos.reset_index(drop=True)
        mergechrompos = pd.concat([mergechrompos, pd_epiall], axis=1)
        get_columns = ['triplettoken','postoken','gestoken','rt','mut_type']+pd_epiall.columns.to_list()

        token_data = mergechrompos[get_columns]

        SNVonly = token_data.loc[token_data['mut_type']=='SNV']
        SNVonly = SNVonly.drop(columns=['mut_type'])
        MNVonly = token_data.loc[token_data['mut_type']=='MNV']
        MNVonly = MNVonly.drop(columns=['mut_type'])
        indelonly = token_data.loc[token_data['mut_type']=='indel']
        indelonly = indelonly.drop(columns=['mut_type'])
        MEISVonly = token_data.loc[token_data['mut_type'].isin(['MEI','SV'])]
        Negonly = token_data.loc[token_data['mut_type']=='Normal']
        Negonly = Negonly.drop(columns=['mut_type'])

        SNVonly.to_csv(tokenized_data + pcawg_histology + '/' + 'SNV_' + pcawg_sample + '.csv')
        MNVonly.to_csv(tokenized_data + pcawg_histology + '/' + 'MNV_' + pcawg_sample + '.csv')
        indelonly.to_csv(tokenized_data + pcawg_histology + '/' + 'indel_' + pcawg_sample + '.csv')
        MEISVonly.to_csv(tokenized_data + pcawg_histology + '/' + 'MEISV_' + pcawg_sample + '.csv')
        Negonly.to_csv(pcawg_dir + pcawg_histology + '/' + 'Normal_' + pcawg_sample + '.csv')
        pd_count = pd.DataFrame([len(SNVonly),len(MNVonly),len(indelonly),len(MEISVonly),len(Negonly)])
        pd_count.to_csv(tokenized_data + pcawg_histology + '/' + 'count_' + pcawg_sample + '.csv')

Bladder-TCC


In [32]:
pd_allsamples = []
for i in range(0,len(all_class)):
    all_samples = os.listdir(sourcedir + all_class[i])
    print(all_class[i])
    pcawg_histology = all_class[i]
    for j in range(0,len(all_samples)):
        onerow = (pcawg_histology,all_samples[j])
        pd_allsamples.append(onerow)

Bladder-TCC
Bone-Osteosarc
Bone-Cart
Bone-Epith
Breast-AdenoCA
Breast-LobularCA
Breast-DCIS
Biliary-AdenoCA
Cervix-SCC
Cervix-AdenoCA
Lymph-CLL
Myeloid-MPN
Myeloid-MDS
Myeloid-AML
ColoRect-AdenoCA
Lymph-BNHL
Prost-AdenoCA
Eso-AdenoCA
Stomach-AdenoCA
CNS-GBM
Head-SCC
Kidney-ChRCC
Kidney-RCC
CNS-Oligo
Liver-HCC
Lung-AdenoCA
Lung-SCC
Lymph-NOS
Skin-Melanoma
Ovary-AdenoCA
Panc-AdenoCA
Panc-Endocrine
CNS-Medullo
CNS-PiloAstro
Bone-Leiomyo
Thy-AdenoCA
Uterus-AdenoCA


In [34]:
pd_allsamples = pd.DataFrame(pd_allsamples)

In [36]:
pd_allsamples.columns = ['tumour_type','sample']

In [38]:
pd_allsamples.to_csv('all_tumoursamples.tsv',sep='\t')